# Enrich Housing Data with FRED API
This notebook pulls Unemployment Rate and Median Household Income from FRED
and joins it with your housing dataset by state and date.

https://fred.stlouisfed.org/

In [2]:
# !pip install fredapi

In [3]:
from fredapi import Fred
import pandas as pd
import matplotlib.pyplot as plt
import time
import json

In [4]:
with open('../keys/keys.json') as fi:
    credentials = json.load(fi)

In [5]:
api_key_fred = credentials['api_key_fred']

In [6]:
fred = Fred(api_key=api_key_fred)

In [7]:
# Load the filtered housing dataset
df = pd.read_csv('../data/processed/01_housing_data_merged.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df.loc[df['RegionName'] == 'United States', 'StateName'] = 'US'
df.head()

,RegionID,RegionName,RegionType,StateName,SizeRank,Date,ZHVI_AllHomes,ZHVI_SingleFamily,ZHVI_Condo,SalesCount_SFR,Inventory_SFR,Year,Month
0,102001,United States,country,US,0,2018-01-31,222399.430599,221704.438544,232520.632007,248552.0,1206651.0,2018,1
1,394913,"New York, NY",msa,NY,1,2018-01-31,468289.939148,458621.808084,496261.053175,10754.0,49048.0,2018,1
2,753899,"Los Angeles, CA",msa,CA,2,2018-01-31,613037.032936,637911.684582,469858.117823,5302.0,16116.0,2018,1
3,394463,"Chicago, IL",msa,IL,3,2018-01-31,227029.700914,231794.550568,190146.965317,5550.0,27301.0,2018,1
4,394514,"Dallas, TX",msa,TX,4,2018-01-31,238849.482465,240010.957031,163186.511511,5077.0,22001.0,2018,1


### Unemployment

In [9]:
# Get unique state codes from your dataset
state_codes = df['StateName'].dropna().unique()

# Build the income series dictionary
unemployment_series = {
    code: f"{code}UR" for code in state_codes
}
unemployment_series['US'] = 'UNRATE'  # Add national series

In [10]:
# Download unemployment data for each state and build a DataFrame
unemployment_dfs = []
for state_code, series_id in unemployment_series.items():
    data = fred.get_series(series_id)
    df_series = data.reset_index()
    df_series.columns = ['Date', 'Unemployment_Rate']
    df_series['Date'] = pd.to_datetime(df_series['Date'])
    df_series['StateName'] = state_code
    unemployment_dfs.append(df_series)
    time.sleep(0.5)

unemployment_all = pd.concat(unemployment_dfs)
unemployment_all['Year'] = unemployment_all['Date'].dt.year
unemployment_all['Month'] = unemployment_all['Date'].dt.month
unemployment_all = unemployment_all.drop_duplicates(['StateName', 'Year', 'Month'])
unemployment_all.head()

,Date,Unemployment_Rate,StateName,Year,Month
0,1948-01-01,3.4,US,1948,1
1,1948-02-01,3.8,US,1948,2
2,1948-03-01,4.0,US,1948,3
3,1948-04-01,3.9,US,1948,4
4,1948-05-01,3.5,US,1948,5


In [11]:
# Merge with unemployment data
df_unemployment = df.merge(
    unemployment_all[['StateName', 'Year', 'Month', 'Unemployment_Rate']],
    how='left',
    on=['StateName', 'Year', 'Month']
)

# df_merged[['RegionName', 'Date', 'Unemployment_Rate']].head(100)
df_unemployment.head()

,RegionID,RegionName,RegionType,StateName,SizeRank,Date,ZHVI_AllHomes,ZHVI_SingleFamily,ZHVI_Condo,SalesCount_SFR,Inventory_SFR,Year,Month,Unemployment_Rate
0,102001,United States,country,US,0,2018-01-31,222399.430599,221704.438544,232520.632007,248552.0,1206651.0,2018,1,4.0
1,394913,"New York, NY",msa,NY,1,2018-01-31,468289.939148,458621.808084,496261.053175,10754.0,49048.0,2018,1,4.5
2,753899,"Los Angeles, CA",msa,CA,2,2018-01-31,613037.032936,637911.684582,469858.117823,5302.0,16116.0,2018,1,4.4
3,394463,"Chicago, IL",msa,IL,3,2018-01-31,227029.700914,231794.550568,190146.965317,5550.0,27301.0,2018,1,4.5
4,394514,"Dallas, TX",msa,TX,4,2018-01-31,238849.482465,240010.957031,163186.511511,5077.0,22001.0,2018,1,4.1


### Median Income

In [13]:
# Get unique state codes from your dataset
state_codes = df_unemployment['StateName'].dropna().unique()

# Build the income series dictionary
income_series = {
    code: f"MEHOINUS{code}A672N" for code in state_codes
}
income_series['US'] = 'MEHOINUSA672N'  # Add national series

# print(income_series)

In [14]:
# Get income data from FRED
income_dfs = []
for state_code, series_id in income_series.items():
    try:
        data = fred.get_series(series_id)
        df_income = data.reset_index()
        df_income.columns = ['Date', 'Median_Income']
        df_income['Date'] = pd.to_datetime(df_income['Date'])
        df_income['Year'] = df_income['Date'].dt.year
        df_income['StateName'] = state_code
        income_dfs.append(df_income)
        time.sleep(0.5)
    except Exception as e:
        print(f"Error for {state_code}: {e}")

income_all = pd.concat(income_dfs)
income_all = income_all.drop_duplicates(['StateName', 'Year'])
income_all.head()

,Date,Median_Income,Year,StateName
0,1984-01-01,58930.0,1984,US
1,1985-01-01,60050.0,1985,US
2,1986-01-01,62280.0,1986,US
3,1987-01-01,63060.0,1987,US
4,1988-01-01,63530.0,1988,US


In [15]:
df_income = df_unemployment.merge(
    income_all[['StateName', 'Year', 'Median_Income']],
    how='left',
    on=['StateName', 'Year']
)

df_income.head()

,RegionID,RegionName,RegionType,StateName,SizeRank,Date,ZHVI_AllHomes,ZHVI_SingleFamily,ZHVI_Condo,SalesCount_SFR,Inventory_SFR,Year,Month,Unemployment_Rate,Median_Income
0,102001,United States,country,US,0,2018-01-31,222399.430599,221704.438544,232520.632007,248552.0,1206651.0,2018,1,4.0,75790.0
1,394913,"New York, NY",msa,NY,1,2018-01-31,468289.939148,458621.808084,496261.053175,10754.0,49048.0,2018,1,4.5,80700.0
2,753899,"Los Angeles, CA",msa,CA,2,2018-01-31,613037.032936,637911.684582,469858.117823,5302.0,16116.0,2018,1,4.4,84560.0
3,394463,"Chicago, IL",msa,IL,3,2018-01-31,227029.700914,231794.550568,190146.965317,5550.0,27301.0,2018,1,4.5,84140.0
4,394514,"Dallas, TX",msa,TX,4,2018-01-31,238849.482465,240010.957031,163186.511511,5077.0,22001.0,2018,1,4.1,71720.0


### Population

In [17]:
state_codes = df_income['StateName'].dropna().unique()

# Create population series dictionary for each state
population_series = {
    code: f"{code}POP" for code in state_codes
}
population_series['US'] = 'POPTHM'  # National monthly estimate

# print(population_series)

In [18]:
# Get population data from FRED
population_dfs = []


for state_code, series_id in population_series.items():
    try:
        # print(f"Fetching {series_id} for {state_code}")
        series = fred.get_series(series_id)
        df_pop = series.reset_index()
        df_pop.columns = ['Date', 'Population']
        df_pop['Date'] = pd.to_datetime(df_pop['Date'])
        df_pop['Year'] = df_pop['Date'].dt.year
        df_pop['StateName'] = state_code
        population_dfs.append(df_pop)
        time.sleep(0.5)
    except Exception as e:
        print(f"Error for {state_code} ({series_id}): {e}")

population_all = pd.concat(population_dfs)
population_all = population_all.drop_duplicates(['StateName', 'Year'])
population_all.head()

,Date,Population,Year,StateName
0,1959-01-01,175818.0,1959,US
12,1960-01-01,178925.0,1960,US
24,1961-01-01,182404.0,1961,US
36,1962-01-01,185347.0,1962,US
48,1963-01-01,188113.0,1963,US


In [19]:
# Merge with main DataFrame
df_population = df_income.merge(
    population_all[['StateName', 'Year', 'Population']],
    how='left',
    on=['StateName', 'Year']
)

df_population.head()

,RegionID,RegionName,RegionType,StateName,SizeRank,Date,ZHVI_AllHomes,ZHVI_SingleFamily,ZHVI_Condo,SalesCount_SFR,Inventory_SFR,Year,Month,Unemployment_Rate,Median_Income,Population
0,102001,United States,country,US,0,2018-01-31,222399.430599,221704.438544,232520.632007,248552.0,1206651.0,2018,1,4.0,75790.0,327969.000
1,394913,"New York, NY",msa,NY,1,2018-01-31,468289.939148,458621.808084,496261.053175,10754.0,49048.0,2018,1,4.5,80700.0,19544.098
2,753899,"Los Angeles, CA",msa,CA,2,2018-01-31,613037.032936,637911.684582,469858.117823,5302.0,16116.0,2018,1,4.4,84560.0,39437.463
3,394463,"Chicago, IL",msa,IL,3,2018-01-31,227029.700914,231794.550568,190146.965317,5550.0,27301.0,2018,1,4.5,84140.0,12724.685
4,394514,"Dallas, TX",msa,TX,4,2018-01-31,238849.482465,240010.957031,163186.511511,5077.0,22001.0,2018,1,4.1,71720.0,28624.564


In [20]:
# Ensure 'Date' is datetime
df_population['Date'] = pd.to_datetime(df_population['Date'])

fred_cols = ['Unemployment_Rate', 'Median_Income', 'Population']

valid_fred_dates = df_population.dropna(subset=fred_cols).groupby('Date').size()

last_valid_fred_date = valid_fred_dates.index.max()

print(f"Last valid date with complete FRED data: {last_valid_fred_date.date()}")

# Для обрізання датасету (необовʼязково)
df_fred_filtered = df_population[df_population['Date'] <= last_valid_fred_date].copy()

# Перевіримо зміну розміру
print(f"Rows before: {df_population.shape[0]}")
print(f"Rows after:  {df_fred_filtered.shape[0]}")


Last valid date with complete FRED data: 2023-12-31
Rows before: 4437
Rows after:  3672


In [21]:
# Save merged DataFrame
df_fred_filtered.to_csv('../data/processed/02_housing_data_enriched_fred.csv', index=False)

print("File successfully saved to 'data/processed/02_housing_data_enriched_fred.csv'")

File successfully saved to 'data/processed/02_housing_data_enriched_fred.csv'
